<a href="https://colab.research.google.com/github/aditya12/adityarepo/blob/WORK/AUTOMATIC1111_SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [stable-diffusion-art.com](https://stable-diffusion-art.com) - Check out latest tutorials.

### [Guide to use this notebook](https://stable-diffusion-art.com/automatic1111-colab/) - Leave comment if you have questions.

#### <b>Become a member to support the maintenance of this notebook.</b>

[<img src="https://stable-diffusion-art.com/wp-content/uploads/2023/10/see_member_benefit_button300.png" width=200>](https://stable-diffusion-art.com/member)

This notebook launches [AUTOMATIC1111 Stable Diffusion Web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)


## Instruction
In the next cell...
1. Review username and password.
2. Check the models you want to load.
3. Click the Play button on left to start.
4. Follow the `gradio.live` or `ngrok.io` link to start using AUTOMATIC1111.


## Update
- 01/12/2025: Updated Anything v3 model link.
- 01/11/2025: Add Inpaint Anything
- 01/11/2025: Downgraded diffusers for Inpaint Anything
- 01/09/2025: Fixed reactor error
- 12/19/2024: Fixed pydantic error







In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="orange">Stable diffusion</font>
#@markdown #### Follow the URL link **`https://xxxxxxx.gradio.live`** or **`https://xxxxx.ngrok-free.app`**to launch the app.
#@markdown See version names <a href="https://github.com/AUTOMATIC1111/stable-diffusion-webui/releases" target="_blank">here</a>
output_path = 'AI_PICS' #@param {type:"string"}
username='a' #@param {type:"string"}
password='a' #@param {type:"string"}
NGROK ='' #@param {type: "string"}
#@markdown Get NGROK key [here](https://dashboard.ngrok.com/get-started/your-authtoken). You can also use secrets.
version='v1.9.3' #@param {type: "string"}
#@markdown Save images, settings, Lora models, embeddings, load models in Google Drive.

#@markdown ## Only check the models you are going to use:
#@markdown ### v1.5 models:
v1_5_model = True #@param{type: "boolean"}
v1_5_inpainting_model = False #@param{type: "boolean"}
F222_model = False #@param{type: "boolean"}
Realistic_Vision_model = True #@param{type: "boolean"}
Realistic_Vision_Inpainting_model = False #@param{type: "boolean"}
DreamShaper_model = True #@param{type: "boolean"}
DreamShaper_Inpainting_model = False #@param{type: "boolean"}
OpenJourney_model = False #@param{type: "boolean"}
Anything_v3_model = False #@param{type: "boolean"}
Inkpunk_Diffusion_model = False #@param{type: "boolean"}
instruct_pix2pix_model = False #@param{type: "boolean"}

#@markdown ### v2 models:
v2_1_768_model = False #@param{type: "boolean"}
v2_depth_model = False #@param{type: "boolean"}

#@markdown ### SDXL models
SDXL_1 = False #@param{type: "boolean"}
JuggernautXL_v8 = False #@param{type: "boolean"}


#@markdown ### Extensions:
ControlNet = False #@param{type: "boolean"}
__ControlNet_SD_1_5_Models = False #@param{type: "boolean"}
__ControlNet_SDXL_Models = False #@param{type: "boolean"}
__ControlNet_IP_Adapter_Models = False #@param{type: "boolean"}
Deforum = False #@param{type: "boolean"}
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}
ADetailer = False #@param{type: "boolean"}
AnimateDiff = False #@param{type: "boolean"}
text2video = False #@param{type: "boolean"}
Reactor = False #@param{type: "boolean"}
Inpaint_Anything = False #@param{type: "boolean"}
Infinite_Image_Browser = True #@param{type: "boolean"}
Aspect_Ratio_Helper = True #@param{type: "boolean"}

#@markdown ### Install models from URL (Huggingface or Civitai. Right-click download button and copy link. Separate multiple links with comma).
Checkpoint_models_from_URL = '' #@param {type: "string"}
Lora_models_from_URL = '' #@param {type: "string"}
Civitai_API_Key = '' #@param {type: "string"}
#@markdown Create API key [here](https://civitai.com/user/account). You can also use secrets.

#@markdown ### Install extensions from URL (separate them with comma).
Extensions_from_URL = '' #@param {type: "string"}

#@markdown ### Extra Web-UI arguments
Extra_arguments = '--disable-model-loading-ram-optimization --opt-sdp-attention' #@param {type: "string"}

CLEAR = True #@param{type: "boolean"}

def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen
        return urlopen(url_or_path)
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url, rename = None):
  url = url.strip()
  url = url.rstrip('/')
  if 'huggingface.co' in url:
    if rename:
      filename = rename
    else:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if rename:
      if '?' in url:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}&token={Civitai_API_Key}" -o {rename}
      else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={Civitai_API_Key}" -o {rename}
    else:
      if '?' in url:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}&token={Civitai_API_Key}"
      else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={Civitai_API_Key}"

def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if v1_5_model:
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if v1_5_inpainting_model:
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
  if F222_model:
    downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')

  if Realistic_Vision_model:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors')
  if Realistic_Vision_Inpainting_model:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1-inpainting.safetensors')

  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')
  if DreamShaper_Inpainting_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors')

  if OpenJourney_model:
    downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
  if Anything_v3_model:
    downloadModel('https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0-pruned-fp16.safetensors')
  if Inkpunk_Diffusion_model:
    downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')
  if instruct_pix2pix_model:
    downloadModel('https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt')

  if v2_1_768_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.yaml

  if SDXL_1:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')

  if JuggernautXL_v8:
    downloadModel('https://civitai.com/api/download/models/288982')


  if Checkpoint_models_from_URL:
      for m in Checkpoint_models_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)

  if Lora_models_from_URL:
      for m in Lora_models_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Lora
        downloadModel(m)


  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Python 3.10 detected. Skip installation.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py


def link_files(source, dest):
  '''Create symlinks for all files in the source folder to the dest folder
  Args:
    source: Absolute path of the source folder
    dest: Absolute path of the destination folder
  '''
  if not os.path.exists(source):
    !mkdir -p {source}
  if not os.path.exists(dest):
    !mkdir -p {dest}
  model_files = glob.glob(source + '/*')
  %cd {dest}
  for f in model_files:
    print(f'Linking model {f} in {dest}')
    !ln -s {f}

def initGoogleDrive(output_path, A1111_local_path):
  # output_path: E.g. AI_PICS
  # A1111_local_path: e.g. /content/stable-diffusion-webui
  A1111_gdrive_path = f"{output_path}/A1111"
  models_gdrive_path = f"{output_path}/models"

  if not os.path.exists(output_path):
    !mkdir -p {output_path}

  if not os.path.exists(A1111_gdrive_path):
    !mkdir -p {A1111_gdrive_path}

  # Use config files in google drive
  # ui-config.json
  if not os.path.exists(A1111_gdrive_path + '/ui-config.json'):
    print("Create new ui-config.json file.")
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {A1111_gdrive_path + '/ui-config.json'}
  !ln -s {A1111_gdrive_path + '/ui-config.json'} {A1111_local_path}

  # Config.json
  if not os.path.exists(A1111_gdrive_path + '/config.json'):
    print("Create new config.json file.")
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {A1111_gdrive_path + '/config.json'}
  !ln -s {A1111_gdrive_path + '/config.json'} {A1111_local_path}

  # styles.csv
  if not os.path.exists(A1111_gdrive_path + '/styles.csv'):
    print("Create new styles.csv file.")
    !wget https://raw.githubusercontent.com/sagiodev/stablediffusion_webui/master/styles.csv -O {A1111_gdrive_path + '/styles.csv'}
  !ln -s {A1111_gdrive_path + '/styles.csv'} {A1111_local_path}

  # save parameter file in google drive
  %cd {A1111_local_path}
  if not os.path.exists(A1111_gdrive_path + '/params.txt'):
    !touch {A1111_gdrive_path + '/params.txt'}
  !ln -s {A1111_gdrive_path}/params.txt

  # create image output folder
  %cd {A1111_local_path}
  if not os.path.exists(f'{output_path}/outputs'):
    !mkdir -p {output_path}/outputs
  !ln -s {output_path}/outputs

  # files
  model_path_map = {
      f'{models_gdrive_path}/embeddings': f'{A1111_local_path}/embeddings',
      f'{models_gdrive_path}/animatediff': f'{A1111_local_path}/models/animatediff',
      f'{models_gdrive_path}/clip': f'{A1111_local_path}/models/clip',
      f'{models_gdrive_path}/clip_vision': f'{A1111_local_path}/models/clip_vision',
      f'{models_gdrive_path}/ControlNet': f'{A1111_local_path}/models/ControlNet',
      f'{models_gdrive_path}/ESRGAN': f'{A1111_local_path}/models/ESRGAN',
      f'{models_gdrive_path}/hypernetworks': f'{A1111_local_path}/models/hypernetworks',
      f'{models_gdrive_path}/insightface': f'{A1111_local_path}/models/insightface',
      f'{models_gdrive_path}/instantid': f'{A1111_local_path}/models/instantid',
      f'{models_gdrive_path}/layer_model': f'{A1111_local_path}/models/layer_model',
      f'{models_gdrive_path}/LDSR': f'{A1111_local_path}/models/LDSR',
      f'{models_gdrive_path}/liveportrait': f'{A1111_local_path}/models/liveportrait',
      f'{models_gdrive_path}/Lora': f'{A1111_local_path}/models/Lora',
      f'{models_gdrive_path}/reactor': f'{A1111_local_path}/models/reactor',
      f'{models_gdrive_path}/Stable-diffusion': f'{A1111_local_path}/models/Stable-diffusion',
      f'{models_gdrive_path}/unet': f'{A1111_local_path}/models/unet',
      f'{models_gdrive_path}/VAE': f'{A1111_local_path}/models/VAE',
      f'{models_gdrive_path}/VAE-approx': f'{A1111_local_path}/models/VAE-approx',
  }

  # Create symbolic links for all models
  for model_path in model_path_map:
    link_files(model_path, model_path_map[model_path])


def installControlNet():
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
  !pip install -r requirements.txt

  %cd {root}/stable-diffusion-webui/models/ControlNet
  if __ControlNet_SD_1_5_Models:
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    #downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    #downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')

  if __ControlNet_SDXL_Models:
    downloadModel('https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_qrpattern.safetensors')
    downloadModel('https://huggingface.co/xinsir/controlnet-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors', rename='diffusion_xl_openpose.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')


  if __ControlNet_IP_Adapter_Models:
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin')
    !pip install insightface

  print("ControlNet install completed.")

def installDeforum():
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {root}/stable-diffusion-webui/extensions/deforum
  #!cd {root}/stable-diffusion-webui/extensions/deforum; git checkout c42834645805e0f26172888b29f5a9210063db14
  !apt-get install libvulkan1

def installRegionalPrompter():
  !git clone https://github.com/hako-mikan/sd-webui-regional-prompter {root}/stable-diffusion-webui/extensions/sd-webui-regional-prompter


def installUltimateSDUpscale():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111

def installOpenPoseEditor():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/huchenlei/sd-webui-openpose-editor

def installADetailer():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Bing-su/adetailer

def installAnimateDiff():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/continue-revolution/sd-webui-animatediff
  # !git checkout b192a2551a5ed66d4a3ce58d5d19a8872abc87ca
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt')
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt')
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt')

def install_Infinite_Image_Browser():
  installExtensionsFromURL('https://github.com/zanllp/sd-webui-infinite-image-browsing')
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-infinite-image-browsing
  !sed 's/IIB_SECRET_KEY=/IIB_SECRET_KEY=SDA/' .env.example > .env


def installtext2video():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/kabachuha/sd-webui-text2video
  %mkdir -p {root}/stable-diffusion-webui/models/text2video/t2v
  %cd {root}/stable-diffusion-webui/models/text2video/t2v
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/VQGAN_autoencoder.pth')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/configuration.json')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/open_clip_pytorch_model.bin')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/text2video_pytorch_model.pth')

def installReactor():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Gourieff/sd-webui-reactor
  !pip install insightface
  !pip install onnxruntime-gpu

def installAspectRatioHelper():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/alemelis/sd-webui-ar
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-ar
  !wget https://raw.githubusercontent.com/sagiodev/stablediffusion_webui/master/resolutions.txt


def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/stable-diffusion-webui/repositories/k-diffusion;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/taming-transformers;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/CodeFormer;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/BLIP;git config core.filemode false

def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/stable-diffusion-webui/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/stable-diffusion-webui/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/stable-diffusion-webui/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/stable-diffusion-webui/repositories/BLIP


def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    print("Cloning extension from URL: %s"%url)
    !git clone {url}

def lowRamPatch():
  print('Apply lowram patch')
  !sed -i 's/dict()))$/dict())).cuda()/g'  {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py

def searchAndReplace(filePath, orignalStr, newStr):
  orignalStr = orignalStr.replace('/', '\/')
  newStr = newStr.replace('/', '\/')
  !sed -i 's/{orignalStr}/{newStr}/g' {filePath}


def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/stable-diffusion-webui/repositories


###################################
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/' + output_path
root = '/content/'

# Get secrets
from google.colab import userdata
try:
  Civitai_API_Key = userdata.get('CIVITAI_API_KEY')
  print('Used CIVITAI_API_KEY secret.')
except:
  pass
try:
  NGROK = userdata.get('NGROK')
  print('Used NGROK secret.')
except:
  pass


# Update to python 3.10
updatePython()

# Clone A1111
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# A1111 first launch
%cd {root}/stable-diffusion-webui
!git checkout -f {version}
!COMMANDLINE_ARGS="--exit"  python launch.py

# Setup Google Drive folders and links
initGoogleDrive(output_path, f"{root}/stable-diffusion-webui")



if ControlNet:
  installControlNet()

if Deforum:
  installControlNet() # Deforum needs controlnet to work now
  installDeforum()

if Regional_Prompter:
  installRegionalPrompter()

if Ultimate_SD_Upscale:
  installUltimateSDUpscale()

if Openpose_Editor:
  installOpenPoseEditor()

if ADetailer:
  installADetailer()

if AnimateDiff:
  installAnimateDiff()

if text2video:
  installtext2video()

if Reactor:
  installReactor()

if Aspect_Ratio_Helper:
  installAspectRatioHelper()

if Infinite_Image_Browser:
  install_Infinite_Image_Browser()

if Inpaint_Anything:
  installExtensionsFromURL('https://github.com/Uminosachi/sd-webui-inpaint-anything')


if Extensions_from_URL:
  installExtensionsFromURL(Extensions_from_URL)

# downgrade httpx to avoid TypeError: AsyncConnectionPool.__init__() got an unexpected keyword argument 'socket_options'
!pip install httpx==0.24.1

# temp solution for ImportError: cannot import name 'computed_field' from 'pydantic
!pip install wandb==0.15.12
!pip install pydantic==1.10.11

# downgrade diffusers for inpaint anything
!pip install diffusers==0.29.2

download_models()

# clear output
if CLEAR:
  clear()



%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue'

if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
if username and password:
  args += f' --gradio-auth "{username}":"{password}" '
args+= ' '+Extra_arguments
print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}